# spannerlog Magic

In [ ]:
#| default_exp magic

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from typing import Optional
import logging
logger = logging.getLogger(__name__)

from IPython.core.magic import (Magics, magics_class, line_cell_magic)
from singleton_decorator import singleton
from spannerlib.session import Session

In [ ]:
#| export
@singleton
class MagicSession():
    def __init__(self):
        self.session = Session()
    def set(self,sess):
        self.session = sess

# TODO add getmagic and setmagic functions and make them the public interface

In [ ]:
sess_1 = Session()
sess_1

<spannerlib.session.Session>

In [ ]:
MagicSession().set(sess_1)

In [ ]:
MagicSession().session

<spannerlib.session.Session>

In [ ]:
#| export
@magics_class
class spannerlogMagic(Magics):
    @line_cell_magic
    def spannerlog(self, line: str, cell: Optional[str] = None) -> None:
        # import locally to prevent circular import issues
        magic_session = MagicSession().session

        code = cell if cell else line

        _ = magic_session.export(code,display_results=True)

def load_ipython_extension(ipython):
    ipython.register_magics(spannerlogMagic)

In [ ]:
try:
    load_ipython_extension(get_ipython())
    logger.warning("spannerlog magic loaded")
except (AttributeError, ImportError):
    pass

spannerlog magic loaded


In [ ]:
%%spannerlog
new A(int)
A(1)
?A(X)

'?A(X)'

,X
0,1


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     